
# Phase 4: Modeling

In this phase, we perform model training and evaluation as part of the CRISP-DM process. 
We will establish a baseline, explore various model families, and optimize their performance using cross-validation and hyperparameter tuning. 



## Naive Baseline Evaluation

The naive baseline prediction uses the mean of the target variable `Chance of Admit` to predict all entries. 
- **Baseline RMSE:** 0.0951



## Ridge Regression

We employ Ridge Regression, optimizing the hyperparameter `alpha` using Optuna's hyperparameter optimization.

```python
# Setup cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Objective function for hyperparameter tuning
def ridge_objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-4, 100)
    model = Ridge(alpha=alpha)
    scores = cross_val_score(model, data.drop(columns='Chance of Admit'), data['Chance of Admit'], 
                             cv=kf, scoring='neg_mean_squared_error')
    return -np.sqrt(-scores.mean())

# Create Optuna study for Ridge Regression
study_ridge = optuna.create_study(direction='minimize', study_name='Ridge Regression')
study_ridge.optimize(ridge_objective, n_trials=50)

# Best hyperparameters and score
best_params_ridge = study_ridge.best_params
best_score_ridge = study_ridge.best_value
print(f"Best Params: {best_params_ridge}, Best RMSE: {best_score_ridge}")
```

- **Best Hyperparameters:** `alpha = {best_params_ridge['alpha']}`
- **Best Score (RMSE):** {best_score_ridge}



## Gradient Boosting Model

The Gradient Boosting Model (GBM) was trained using Optuna to find the best hyperparameters:

```python
# Objective function for GBM hyperparameter tuning
def gbm_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    model = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=42)
    scores = cross_val_score(model, data.drop(columns='Chance of Admit'), data['Chance of Admit'], cv=kf, scoring='neg_mean_squared_error')
    return -np.sqrt(-scores.mean())

# Create Optuna study for GBM
study_gbm = optuna.create_study(direction='minimize', study_name='GBM')
study_gbm.optimize(gbm_objective, n_trials=50)

# Best hyperparameters and score
best_params_gbm = study_gbm.best_params
best_score_gbm = study_gbm.best_value
print(f"Best Params: {best_params_gbm}, Best RMSE: {best_score_gbm}")
```

- **Best Hyperparameters**:
  - `n_estimators`: {best_params_gbm['n_estimators']}
  - `max_depth`: {best_params_gbm['max_depth']}
  - `learning_rate`: {best_params_gbm['learning_rate']}
- **Best Score (RMSE):** {best_score_gbm}



## Neural Network

The Neural Network model was trained using Keras and optimized using Optuna:

```python
# Objective function for Neural Network optimization
def nn_objective(trial):
    units_layer1 = trial.suggest_int('units_layer1', 16, 256, log=True)
    units_layer2 = trial.suggest_int('units_layer2', 16, 256, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    # Build model
    model = keras.Sequential([
        layers.Dense(units_layer1, activation='relu', input_shape=(data.shape[1] - 1,)),
        layers.Dropout(dropout_rate),
        layers.Dense(units_layer2, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    
    # Compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='mse')
    
    # Train model
    model.fit(data.drop(columns='Chance of Admit'), data['Chance of Admit'],
              epochs=50, batch_size=32, verbose=0, validation_split=0.2)

    # Evaluate model
    score = model.evaluate(data.drop(columns='Chance of Admit'), data['Chance of Admit'], verbose=0)
    return np.sqrt(score)

# Create Optuna study for Neural Network
study_nn = optuna.create_study(direction='minimize', study_name='Neural Network')
study_nn.optimize(nn_objective, n_trials=20)

# Best hyperparameters and score
best_params_nn = study_nn.best_params
best_score_nn = study_nn.best_value
print(f"Best Params: {best_params_nn}, Best RMSE: {best_score_nn}")
```

- **Best Hyperparameters**:
  - `units_layer1`: {best_params_nn['units_layer1']}
  - `units_layer2`: {best_params_nn['units_layer2']}
  - `dropout_rate`: {best_params_nn['dropout_rate']}
  - `learning_rate`: {best_params_nn['learning_rate']}
- **Best Score (RMSE):** {best_score_nn}
